In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()

    def call(self, targets, logits, sample_weights=None):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        loss = self.loss_fn(targets, logits, sample_weights)
        self.add_loss(loss)

        # Log accuracy as a metric and add it
        # to the layer using `self.add_metric()`.
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name="accuracy")

        # Return the inference-time prediction tensor (for `.predict()`).
        return tf.nn.softmax(logits)

layer = LogisticEndpoint()

targets = tf.ones((2, 2))
logits = tf.ones((2, 2))
y = layer(targets, logits)

print("layer.metrics:", layer.metrics)
print("current accuracy value:", float(layer.metrics[0].result()))
inputs = keras.Input(shape=(3,), name="inputs")
targets = keras.Input(shape=(10,), name="targets")
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name="predictions")(logits, targets)

model = keras.Model(inputs=[inputs, targets], outputs=predictions)
model.compile(optimizer="adam")

data = {
    "inputs": np.random.random((3, 3)),
    "targets": np.random.random((3, 10)),
}
model.fit(data)

layer.metrics: [<tensorflow.python.keras.metrics.BinaryAccuracy object at 0x000002BFA113A1D0>]
current accuracy value: 1.0
Train on 3 samples
3/3 [==============================] - 1s 293ms/sample - loss: 0.9531 - binary_accuracy: 0.0000e+00


# Practical Implementation of InceptionV3

In [2]:
#tensorflow verion 2.0 with keras 2.3 version

#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers

import tensorflow as tf
from tensorflow import keras
import numpy as np

#from keras.models import Model
#from keras.layers import Input
#from keras.layers import Conv2D
#from keras.layers import MaxPooling2D
#from keras.layers.merge import concatenate
#from keras.utils import plot_model

In [3]:
#inception naive version

def inception_module(x, f1, f2, f3):
	# 1x1 conv
	conv1 =  keras.layers.Conv2D(f1, (1,1), padding='same', activation='relu')(x)
	# 3x3 conv
	conv3 = keras.layers.Conv2D(f2, (3,3), padding='same', activation='relu')(x)
	# 5x5 conv
	conv5 = keras.layers.Conv2D(f3, (5,5), padding='same', activation='relu')(x)
	# 3x3 max pooling
	pool = keras.layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(x)
	# concatenate filters
	out = keras.layers.merge.concatenate([conv1, conv3, conv5, pool])
	return out

In [4]:


img_input = keras.Input(shape=(299, 299, 3))
classes=1000
#WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.5/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
#WEIGHTS_PATH = 'inception_v3_weights_tf_dim_ordering_tf_kernels.h5'
#WEIGHTS_PATH = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
channel_axis=3

In [18]:
def conv2d_bn(x,filters,num_row,num_col,padding='same',strides=(1, 1)):
   
    x = keras.layers.Conv2D(filters, (num_row, num_col),strides=strides,padding=padding)(x)
    x = keras.layers.BatchNormalization(axis=3, scale=False)(x)
    x = keras.layers.Activation('relu')(x)
    return x

# num_row and num_cols are height and width of filter

# Building a model layer by layer

In [19]:
def inc_block_a(x):    
    branch1x1 = conv2d_bn(x, 64, 1, 1)  # 64 filters of 1*1

    branch5x5 = conv2d_bn(x, 48, 1, 1)  #48 filters of 1*1
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)

    branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = keras.layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool], axis=channel_axis)
    return x

In [20]:
def reduction_block_a(x):  
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2, 2), padding='valid')

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = keras.layers.concatenate([branch3x3, branch3x3dbl, branch_pool],axis=channel_axis)
    return x

In [21]:
# 17 x 17 x 768
def inc_block_b(x):
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)

    branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1),padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = keras.layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool], axis=channel_axis)
    return x

In [22]:
# mixed 8: 8 x 8 x 1280
def reduction_block_b(x): 
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,strides=(2, 2), padding='valid')

    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn( branch7x7x3, 192, 3, 3, strides=(2, 2), padding='valid')

    branch_pool = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)
    x = keras.layers.concatenate([branch3x3, branch7x7x3, branch_pool], axis=channel_axis)
    return x


In [23]:
def inc_block_c(x):        
        branch1x1 = conv2d_bn(x, 320, 1, 1)

        branch3x3 = conv2d_bn(x, 384, 1, 1)
        branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
        branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
        branch3x3 = keras.layers.concatenate([branch3x3_1, branch3x3_2],axis=channel_axis)

        branch3x3dbl = conv2d_bn(x, 448, 1, 1)
        branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
        branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
        branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
        branch3x3dbl = keras.layers.concatenate([branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

        branch_pool = keras.layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
        branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
        x = keras.layers.concatenate( [branch1x1, branch3x3, branch3x3dbl, branch_pool],axis=channel_axis)
        return x

In [24]:
img_input = keras.Input(shape=(299, 299, 3))  #shape=(None, 299, 299, 3)
  

In [30]:
# input image size: 299 x 299 x 3
x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid') # 149 x 149 x 32
x = conv2d_bn(x, 32, 3, 3, padding='valid')  # 147 x 147 x 32
x = conv2d_bn(x, 64, 3, 3) # 147 x 147 x 64

x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)   # 73  x 73 x 64
x = conv2d_bn(x, 80, 1, 1, padding='valid') # 73 x 73 x 80
x = conv2d_bn(x, 192, 3, 3, padding='valid')  # 71 x 71 x 192
x = keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)  # 35 x 35 x 192


x=inc_block_a(x) #35, 35, 256
x=inc_block_a(x) #35, 35, 256
x=inc_block_a(x) #35, 35, 256

x=reduction_block_a(x) #17, 17, 736

x=inc_block_b(x) #17, 17, 768
x=inc_block_b(x) #17, 17, 768
x=inc_block_b(x) #17, 17, 768
x=inc_block_b(x) #17, 17, 768

x=reduction_block_b(x) #shape=(None, 8, 8, 1280)

x=inc_block_c(x) # shape=(None, 8, 8, 2048) 
x=inc_block_c(x) # shape=(None, 8, 8, 2048) 

x = keras.layers.GlobalAveragePooling2D(name='avg_pool')(x) # shape=(None, 2048)

x = keras.layers.Dense(classes, activation='softmax', name='predictions')(x) #shape=(None, 1000) 



# Create model.
inputs = img_input
model =  keras.Model(inputs, x, name='inception_v3')
model.summary()


# plot model architecture
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='inception_model_3.png')

Tensor("activation_631/Identity:0", shape=(None, 149, 149, 32), dtype=float32)
Tensor("activation_632/Identity:0", shape=(None, 147, 147, 32), dtype=float32)
Tensor("activation_633/Identity:0", shape=(None, 147, 147, 64), dtype=float32)
Tensor("max_pooling2d_28/Identity:0", shape=(None, 73, 73, 64), dtype=float32)
Tensor("activation_634/Identity:0", shape=(None, 73, 73, 80), dtype=float32)
Tensor("activation_635/Identity:0", shape=(None, 71, 71, 192), dtype=float32)
Tensor("max_pooling2d_29/Identity:0", shape=(None, 35, 35, 192), dtype=float32)
Tensor("concatenate_96/Identity:0", shape=(None, 35, 35, 256), dtype=float32)
Tensor("concatenate_97/Identity:0", shape=(None, 35, 35, 256), dtype=float32)
Tensor("concatenate_98/Identity:0", shape=(None, 35, 35, 256), dtype=float32)
Tensor("concatenate_99/Identity:0", shape=(None, 17, 17, 736), dtype=float32)
Tensor("concatenate_100/Identity:0", shape=(None, 17, 17, 768), dtype=float32)
Tensor("concatenate_101/Identity:0", shape=(None, 17, 17, 

<tf.Tensor 'predictions_5/Identity:0' shape=(None, 1000) dtype=float32>

Using TensorFlow backend.


Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_631 (Conv2D)             (None, 149, 149, 32) 896         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_631 (BatchN (None, 149, 149, 32) 96          conv2d_631[0][0]                 
__________________________________________________________________________________________________
activation_631 (Activation)     (None, 149, 149, 32) 0           batch_normalization_631[0][0]    
_______________________________________________________________________________________

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.